[마케팅] 자동차 시장 세분화
자동차 회사는 새로운 전략을 수립하기 위해 4개의 시장으로 세분화했습니다.

기존 고객 분류 자료를 바탕으로 신규 고객이 어떤 분류에 속할지 예측해주세요!

예측할 값(y): "Segmentation" (1,2,3,4)

평가: Macro f1-score

data: train.csv, test.csv

In [9]:
import numpy as np
import pandas as pd
train = pd.read_csv('train.csv')
# train.info()    #결측치 없음
y = train['Segmentation']
x = train.drop(columns=['Segmentation', 'ID'])
x

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4
1,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6
2,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6
3,Male,Yes,56,No,Artist,0.0,Average,2.0,Cat_6
4,Male,No,32,Yes,Healthcare,1.0,Low,3.0,Cat_6
...,...,...,...,...,...,...,...,...,...
6660,Male,Yes,41,Yes,Artist,0.0,High,5.0,Cat_6
6661,Male,No,35,No,Executive,3.0,Low,4.0,Cat_4
6662,Female,No,33,Yes,Healthcare,1.0,Low,1.0,Cat_6
6663,Female,No,27,Yes,Healthcare,1.0,Low,4.0,Cat_6


In [10]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           6665 non-null   object 
 1   Ever_Married     6665 non-null   object 
 2   Age              6665 non-null   int64  
 3   Graduated        6665 non-null   object 
 4   Profession       6665 non-null   object 
 5   Work_Experience  6665 non-null   float64
 6   Spending_Score   6665 non-null   object 
 7   Family_Size      6665 non-null   float64
 8   Var_1            6665 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 468.8+ KB


In [11]:
X_cat = x.select_dtypes('object')
X_con = x.select_dtypes(exclude='object')

In [26]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
enc = OneHotEncoder(sparse=False)
sc = StandardScaler()

enc.fit(X_cat)
sc.fit(X_con)

X_cat_enc = enc.transform(X_cat)
X_con_sc = sc.transform(X_con)

C:\Users\JYS\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [27]:
X_cat_enc

array([[0., 1., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       ...,
       [1., 0., 1., ..., 0., 1., 0.],
       [1., 0., 1., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [21]:
X_con_sc

array([[-1.30341502, -0.4784302 ,  0.76011295],
       [ 1.42009199, -0.4784302 , -1.20757975],
       [ 1.42009199, -0.7721065 , -0.55168218],
       ...,
       [-0.63766886, -0.4784302 , -1.20757975],
       [-1.00080313, -0.4784302 ,  0.76011295],
       [-0.39557935, -0.7721065 ,  0.10421538]])

In [28]:
import numpy as np
X = np.concatenate([X_cat_enc, X_con_sc], axis = 1)
X

array([[ 0.        ,  1.        ,  1.        , ..., -1.30341502,
        -0.4784302 ,  0.76011295],
       [ 1.        ,  0.        ,  0.        , ...,  1.42009199,
        -0.4784302 , -1.20757975],
       [ 0.        ,  1.        ,  0.        , ...,  1.42009199,
        -0.7721065 , -0.55168218],
       ...,
       [ 1.        ,  0.        ,  1.        , ..., -0.63766886,
        -0.4784302 , -1.20757975],
       [ 1.        ,  0.        ,  1.        , ..., -1.00080313,
        -0.4784302 ,  0.76011295],
       [ 0.        ,  1.        ,  0.        , ..., -0.39557935,
        -0.7721065 ,  0.10421538]])

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [32]:
(X_train.shape, y_train.shape)

((5998, 28), (5998,))

In [54]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=7)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=7)

In [39]:
from sklearn.metrics import f1_score
pred = rf.predict(X_val)
print(f1_score(y_val, pred, average='macro'))


0.4880305589337297


In [48]:
X_test = pd.read_csv('test.csv')
ID = X_test['ID']

In [49]:
X_test = X_test.drop(columns=['ID'])
X_test_cat = X_test.select_dtypes('object')
X_test_con = X_test.select_dtypes(exclude='object')
enc.fit(X_test_cat)
sc.fit(X_test_con)
X_test_cat = enc.transform(X_test_cat)
X_test_con = sc.transform(X_test_con)
X_test = np.concatenate([X_test_cat, X_test_con], axis = 1)

C:\Users\JYS\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [46]:
X_test

array([[ 1.        ,  0.        ,  0.        , ..., -0.44524796,
        -0.76298583, -1.17270206],
       [ 0.        ,  1.        ,  0.        , ..., -0.38557499,
         1.62925795,  0.74238531],
       [ 0.        ,  1.        ,  0.        , ...,  0.92723055,
         2.52634937, -0.53433961],
       ...,
       [ 1.        ,  0.        ,  1.        , ..., -0.50492094,
        -0.46395536, -1.17270206],
       [ 0.        ,  1.        ,  0.        , ...,  0.2111548 ,
        -0.46395536,  1.38074776],
       [ 1.        ,  0.        ,  1.        , ..., -0.02753711,
         1.92828842,  0.10402285]])

In [55]:
res = rf.predict(X_test)
res

array([1, 3, 2, ..., 1, 3, 4], dtype=int64)

In [57]:
output = pd.DataFrame({'ID':ID, 'Segmentation':res})
output.to_csv('output.csv', index=False)